In [1]:
import numpy as np

import cv2 as cv
import cvzone

from IPython.display import clear_output

## Support function

In [2]:
def empty(arg):
    pass

def preprocessing(image:np.ndarray):
    """
    Preprocesses the image: smoothes, finds contours and performs a morphological closure operation.
    """
    threshold1, threshold2 = cv.getTrackbarPos("Threshold1", "Settings"), cv.getTrackbarPos("Threshold2", "Settings")
    
    preprocessing_image = cv.GaussianBlur(image, (5, 5), 3)
    preprocessing_image = cv.Canny(preprocessing_image, threshold1=threshold1, threshold2=threshold2)
    
    kernel = np.ones((3, 3), np.uint8)
    preprocessing_image = cv.dilate(preprocessing_image, kernel, iterations=1)
    preprocessing_image = cv.morphologyEx(preprocessing_image, cv.MORPH_CLOSE, kernel)
    
    return preprocessing_image

def counting_money(contours:list):
    """
    Based on certain contours calculates the amount of money based on the coins in the frame. 
    Threshold values of the areas of detected objects are used for calculation.
    """
    money = 0
    
    if contours_found:
        for contour in contours_found:
            # Get the length of the contour
            peri = cv.arcLength(contour["cnt"], True)
            # Approximate the contour with a given accuracy relative to its length
            approx = cv.approxPolyDP(contour["cnt"], 0.02 * peri, True) 
            
            if len(approx) > 5:
                area = contour["area"]
                
                if area > 1000 and area < 1800:
                    money += 1
                elif area > 1800:
                    money += 2
                    
    return money

## Main

In [3]:
total_money = 0

# Сapturing an image from a webcam
capture = cv.VideoCapture(0)
capture.set(3, 640)
capture.set(4, 480)

# Create window with settings
cv.namedWindow("Settings")
cv.resizeWindow("Settings", 640, 120)
cv.createTrackbar("Threshold1", "Settings", 50, 255, empty)
cv.createTrackbar("Threshold2", "Settings", 250, 255, empty)

while True:
    success, image = capture.read()
    preprocessing_image = preprocessing(image)
    
    contours_image, contours_found = cvzone.findContours(image, preprocessing_image, minArea=20)
    
    total_money = counting_money(contours_found)
    
    stacked_image = cvzone.stackImages([image, contours_image], 2, 1)
    cvzone.putTextRect(stacked_image, f"Money (rub) = {total_money}", (50, 50), colorR=(255, 0, 0))
    cv.imshow("Coin counter", stacked_image)
    
    if cv.waitKey(1) & 0xFf == ord('q'):
        capture.release()
        cv.destroyAllWindows()
        break